In [1]:
from urllib2 import urlopen, Request
from bs4 import BeautifulSoup
from pandas import DataFrame
import pandas as pd
import numpy as np
import re
import os.path 
import datetime 
import sys
import json
import gc


from __future__ import division

import nltk,re, pprint

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

bs = BeautifulSoup(urlopen('http://www.playbill.com/article/broadway-real-estate-com-321997'))
Theatres = DataFrame([{'name': a.getText(), 'url': a['href'], 'address': a.nextSibling[2:]} 
                      for a in bs.find_all("a") if a.has_attr('href') and '/venue/' in a['href']])

Shows = pd.read_csv('Shows.csv')

def getTags(url):
    bs = BeautifulSoup(urlopen('http://www.playbill.com' + url))
    return sorted(set([x.getText() for x in bs.find_all('h5', class_='bsp-bio-sub-text')]))

def getTagsFromBs(bs):
    return sorted(set([x.getText() for x in bs.find_all('h5', class_='bsp-bio-sub-text')]))

def seasonShows(season):
    seasonJson = json.loads(''.join(urlopen('http://www.playbill.com/ajax/loadProdSeason.jsp?year='+str(season)+'&start=0&limit=1000').readlines()))
    result = []
    for row in seasonJson['rows']:

        bs = BeautifulSoup(row)
        if (bs.find('a').has_attr('href')):
            result.append({'url':bs.find('a')['href'], 'title': bs.find('a').getText()})
    return result

def getProductionHtml(url):
    if (not url.startswith('http://www.playbill.com')):
        url = 'http://www.playbill.com' + url

    filename = 'production/' + url.split('/')[-1] + '.html' 

    if (not os.path.isfile(filename)):
        html = urlopen(url).read()
        f = open(filename, "w")
        f.write(html)
        f.close()
        print 'Wrote ' + filename
    else:
        print filename + ' already exists'

def getHtmlForYear(year):
    print 'Year: ' + str(year)
    [getProductionHtml(x) for x in list(Shows[Shows['year']==year]['url'])]
    

In [2]:
tagsForUrl = {}

for year in range(1700,2018):
    print datetime.datetime.now()
    print year,
    for url in list(Shows[Shows['year'] == year]['url']):
        filename = (url + '.html')[1:]
        if (os.path.isfile(filename)):
            f = open(filename)
            bs = BeautifulSoup(f.read())
            f.close()
            tagsForUrl[url] = '/'.join(getTagsFromBs(bs))
            bs.decompose()
            gc.collect() #BeautifulSoup memory leak!!!! :(((
            print ".",
        else:
            print filename + ' not found'
    print
print datetime.datetime.now()

2018-04-05 14:34:28.544574
1700
2018-04-05 14:34:28.549886
1701
2018-04-05 14:34:28.551815
1702
2018-04-05 14:34:28.553278
1703
2018-04-05 14:34:28.554234
1704
2018-04-05 14:34:28.555372
1705
2018-04-05 14:34:28.557303
1706
2018-04-05 14:34:28.558806
1707
2018-04-05 14:34:28.560146
1708
2018-04-05 14:34:28.561346
1709
2018-04-05 14:34:28.562561
1710
2018-04-05 14:34:28.563676
1711
2018-04-05 14:34:28.564821
1712
2018-04-05 14:34:28.565796
1713
2018-04-05 14:34:28.566762
1714
2018-04-05 14:34:28.567983
1715
2018-04-05 14:34:28.569182
1716
2018-04-05 14:34:28.570325
1717
2018-04-05 14:34:28.571499
1718
2018-04-05 14:34:28.572833
1719
2018-04-05 14:34:28.574114
1720
2018-04-05 14:34:28.575621
1721
2018-04-05 14:34:28.576899
1722
2018-04-05 14:34:28.578336
1723
2018-04-05 14:34:28.579448
1724
2018-04-05 14:34:28.580608
1725
2018-04-05 14:34:28.581731
1726
2018-04-05 14:34:28.583170
1727
2018-04-05 14:34:28.584764
1728
2018-04-05 14:34:28.585930
1729
2018-04-05 14:34:28.587556
1730
2018-04-

In [6]:
Shows['tags'] = Shows['url'].apply(lambda url: tagsForUrl[url])

In [7]:
Shows.to_csv('Shows.csv',index=False,encoding='utf-8')

In [ ]:
print datetime.datetime.now()


In [ ]:
[getHtmlForYear(y) for y in range(1700,2018)]

In [ ]:
bss = [BeautifulSoup(urlopen(url)) for url in list(Theatres['url'])]

In [ ]:
nltk.Text(nltk.word_tokenize(''.join(theaterDescriptions))).collocations()

In [ ]:
theaterDescriptions = []

for i in range(39):    
    print i
    
    try:
        bs = bss[i]

        bspBio = bs.find('div', class_='bsp-bio')

        bspBioTitle = bspBio.find('h2', class_='bsp-bio-title').getText()

        print bspBioTitle

        bspBioText = ''
        bspBioFullTextUrl = ''
        if (bspBio.find('div', class_='bsp-bio-text')):
            bspBioText =  bspBio.find('div', class_='bsp-bio-text').getText('\n')
            if (bspBio.find('div', class_='bsp-bio-text').find('a')):
                if (bspBio.find('div', class_='bsp-bio-text').find('a').has_attr('href')):
                    bspBioFullTextUrl = bspBio.find('div', class_='bsp-bio-text').find('a')['href']

        bspBioLists =bspBio.find('div', class_='bsp-bio-lists')
        bspBioListPairs = [(re.sub(':$', '', li.find('span').getText()), li.find('span').nextSibling) for li in bspBioLists.find_all('li')]


        print bspBioText
        theaterDescriptions.append(bspBioText)
        
        print bspBioFullTextUrl
        print bspBioListPairs
    except:
        print("Unexpected error:", sys.exc_info()[0])

In [ ]:
t = DataFrame(theaterDescriptions)
t[0]=t[0].str.strip()
t.to_csv('theaterDescriptions.csv',index=False,encoding='utf-8')


In [ ]:
text = nltk.word_tokenize(theaterDescriptions[10])
nltk.pos_tag(text)

In [ ]:
season2017 = json.loads(''.join(urlopen('http://www.playbill.com/ajax/loadProdSeason.jsp?year=2017&start=0&limit=1000').readlines()))


In [ ]:


Shows = DataFrame([{'year': year, 'url': x['url'], 'title': x['title']}  for year in range(1701,2018) for x in seasonShows(year)])

Shows['title'] = Shows['title'].str.strip()



In [ ]:
Shows.groupby('year').count()[['title']].plot.bar(figsize=(30,5))

In [ ]:
print len(Shows.groupby('url').count())
print len(Shows)

In [ ]:
list(Shows[Shows['url'].str.contains('carousel')]['url'])

In [ ]:
carousels

In [ ]:
url =  '/production/carousel-2017-2018'

url = 'http://www.playbill.com' + url

bs = BeautifulSoup(urlopen(url))

In [ ]:
tags = [getTags(url) for url in list(Shows['url'])]

In [ ]:
getTags('/production/carousel-2017-2018')

In [ ]:
carousels= list(Shows[Shows['url'].str.contains('carousel')]['url'])

In [ ]:
previousShows = 'http://www.playbill.com/ajax/loadVenueShow.jsp?id=00000150-aa9f-d8be-af71-ffbf689b0003&start=0&limit=1000&venue=00000150-aacd-d8be-af71-ffef18860005'

In [ ]:
url='http://www.playbill.com/production/carousel-2017-2018'
bs = BeautifulSoup(urlopen(url))

[json.loads(x['data-bsp-carousel-options'])['themeConfig']['dynamicSlideLoadNextUrl'] for x 
 in bs.find_all('div', class_='gallery-module gallery-module-horizontal')]

In [ ]:
url='http://www.playbill.com/venue/imperial-theatre-vault-0000000201'
bs = BeautifulSoup(urlopen(url))

[json.loads(x['data-bsp-carousel-options'])['themeConfig']['dynamicSlideLoadNextUrl'] for x 
 in bs.find_all('div', class_='gallery-module gallery-module-horizontal')]

In [ ]:
url='http://www.playbill.com/production/carousel-2017-2018'
bs = BeautifulSoup(urlopen(url))

In [ ]:
getProductionHtml('http://www.playbill.com/production/carousel-2017-2018')


In [ ]:
str(urlopen(url))

In [ ]:
url.startswith('http://www.playbill.com/')

In [ ]:
f = open('production/my-fair-lady-uris-theatre-vault-0000011053.html', "r")
bs = BeautifulSoup(f.read())
f.close()

bs

In [ ]:
[json.loads(x['data-bsp-carousel-options'])['themeConfig']['dynamicSlideLoadNextUrl'] for x 
 in bs.find_all('div', class_='gallery-module gallery-module-horizontal')]

In [ ]:
getTagsFromBs(bs)